In [1]:
from urllib.parse import urlencode
from urllib.request import urlretrieve

In [16]:
BASE_URL = "http://history.openweathermap.org/data/2.5/history/city?"
API_KEY = open("api_key.txt").read()

In [21]:
def get_history_weather(params):
    """
    Call OpenWeatherMap historical weather API by coordinates
    Returns JSON file
    """
    query_dict = {"lat": params["lat"],
                  "lon": params["lon"],
                  "type": "hour",
                  "start": params["start"],
                  "end": params["end"],
                  "appid": API_KEY}

    query_str = urlencode(query_dict)

    print(BASE_URL, query_str)
    api_request = urlretrieve(BASE_URL, query_str)
    
    return api_request
    #print(api_request)

In [18]:
params = {"lat": -122,
         "lon": 45,
         "type": "hour",
         "start": 1369728000,
         "end": 1369789200}

In [22]:
get_history_weather(params)

http://history.openweathermap.org/data/2.5/history/city? lat=-122&lon=45&type=hour&start=1369728000&end=1369789200&appid=6bce826503c879f393a3f2f7fd8e35b7


HTTPError: HTTP Error 401: Unauthorized